In [1]:
import opensim as osim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from shutil import copyfile
import os
import lxml.etree as etree

# Function definition

In [121]:
def Update_Controller(participant, ID_file,trial):
    ID_path = f"C:\\Users\\Assila\\Documents\\CEINMS_test\\ElaboratedData\\XmlGenerator\\{participant}\\Trials\\{trial}\\wu_{ID_file}.sto"
    Act_path = f"C:\\Users\\Assila\\Documents\\CEINMS_test\\ElaboratedData\\XmlGenerator\\{participant}\\Trials\\{trial}\\Added_controllers.sto"
    File = pd.read_csv(ID_path, sep='\t', header=6)
    ID=pd.DataFrame([File.time, File.thorax_tilt_moment, File.thorax_list_moment, File.thorax_rotation_moment,
                 File.thorax_tx_force, File.thorax_ty_force, File.thorax_tz_force,
                 File.box_rotX_moment, File.box_rotY_moment, File.box_rotZ_moment,
                 File.box_transX_force, File.box_transY_force, File.box_transZ_force,
                 File.sternoclavicular_r1_moment, File.sternoclavicular_r2_moment, File.sternoclavicular_r3_moment,
                 File.Acromioclavicular_r1_moment, File.Acromioclavicular_r2_moment, File.Acromioclavicular_r3_moment,
                 File.elbow_flexion_moment, File.pro_sup_moment, File.hand_r_Add_moment, File.hand_r_Flex_moment]).transpose()
    ID=ID.rename(index = str, columns={"time":"time", "thorax_tilt_moment":"th_tilt", "thorax_list_moment":"th_list", 
                                "thorax_rotation_moment":"th_rotation", "thorax_tx_force":"th_tx", "thorax_ty_force": "th_ty", 
                                "thorax_tz_force": "th_tz", "box_rotX_moment":"bo_rotX", "box_rotY_moment":"bo_rotY", 
                                "box_rotZ_moment":"bo_rotZ", "box_transX_force":"bo_transX", "box_transY_force":"bo_transY",
                                "box_transZ_force":"bo_transZ", "sternoclavicular_r1_moment":"st_r1", 
                                "sternoclavicular_r2_moment":"st_r2", "sternoclavicular_r3_moment":"st_r3", 
                                "Acromioclavicular_r1_moment":"Ac_r1", "Acromioclavicular_r2_moment":"Ac_r2",
                                "Acromioclavicular_r3_moment":"Ac_r3", "elbow_flexion_moment":"el_flexion", "pro_sup_moment":"pr_sup",
                                "hand_r_Add_moment":"ha_Add", "hand_r_Flex_moment":"ha_Flex"
                               })  
    ID.head()
    ID.shape
    header = list()
    header.append(f"Actuator_{participant}_{trial}\nversion=1\nnRows={ID.shape[0]}\n"
                  f"nColumns={ID.shape[1]}\ninDegrees=no\nendheader\n")
    for col in ID.columns:
        header.append(f"{col}\t")
    header=''.join(header)
    act= ID.to_numpy()
    np.savetxt(Act_path,act, delimiter='\t', header=header, comments='' )

In [125]:
def reduce_model(participant, trial):
    ori_model_path = f"C:\\Users\\Assila\\Documents\\CEINMS_test\\ElaboratedData\\XmlGenerator\\{participant}\\models\\wu_scaled_markers_optCEINMS.osim"
    optimal_forces = [3000, 3000, 3000, 5000, 5000, 5000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 0, 0, 0,
                      1000,1000,1000,3000, 3000, 3000, 5000, 5000, 5000]
    model = osim.Model(ori_model_path)
    
#     Create Controller:
    pre= osim.PrescribedController()
    File_path = f"C:\\Users\\Assila\\Documents\\CEINMS_test\\ElaboratedData\\XmlGenerator\\{participant}\\Trials\\{trial}\\Added_controllers.sto"
    pre.set_controls_file(File_path)
    pre.set_interpolation_method(5)
    pre.upd_ControlFunctions()
    pre.upd_interpolation_method()
#     Create actuators and reduce model to GH muscles:

    ddl = model.getCoordinateSet()
    con_set = model.getForceSet()
    res_s = []
    for mu in range(13):
        con_set.remove(0)
    for i in range(ddl.getSize()):
        ddli = ddl.get(i).getName()
        if ddli =="shoulder_plane"or ddli=="shoulder_ele" or ddli=="shoulder_rotation":
            i=i+3
        else:
            ddl_name = ddli.split("_")[0][0:2]+"_"+ddli.split("_")[-1]
            res = osim.CoordinateActuator(ddli)
            res.setName(ddl_name)
            res.set_optimal_force(optimal_forces[i])
            res.set_min_control(-float('Inf'))
            res.set_max_control(float('Inf'))
            pre.addActuator(res)
            con_set.append(res)
            res_s.append(res)
    model.addController(pre)
    new_osim_path = f"C:\\Users\\Assila\\Documents\\CEINMS_test\\ElaboratedData\\XmlGenerator\\{participant}\\Trials\\{trial}\\Actuated"
    model.setName('Opt_red_'+participant+'_'+trial)
    model.printToXML(new_osim_path+'_'+participant+'_'+trial+'.osim')

In [241]:
def write_xml(file, xml_root_element, file_type, 
              xml_declaration=False, pretty_print=False, encoding='unicode', indent='  '):
    s = etree.tostring(xml_root_element, xml_declaration=xml_declaration, pretty_print=pretty_print, encoding=encoding)
    if pretty_print:
        s = s.replace('  ', indent)
    if file_type == "Trial":
        try:
            s = s.replace('__COLON__', ":")
            while s.find('__RM_BEGIN__') >= 0:
                s = s[:s.find('__RM_BEGIN__')] + s[s.find('__RM_END__') + len('__RM_END__'):]

            with open(file, 'w') as f:
                f.write('<?xml version="1.0" encoding="UTF-8" standalone="no" ?>\n')
                f.write(s)
        except IOError:
            print("Error while writing in log file!")
    elif file_type =="Ext_load":
        try:
            s = s.replace('__COLON__', ":")
            while s.find('__RM_BEGIN__') >= 0:
                s = s[:s.find('__RM_BEGIN__')] + s[s.find('__RM_END__') + len('__RM_END__'):]
            with open(file, 'w') as f:
                f.write('<?xml version="1.0" encoding="UTF-8" ?>\n')
                f.write(s)
        except IOError:
            print("Error while writing in log file!")
    else:
        print("file type can either be Trial or Ext_load")
        

In [262]:
def trial_load(participant, sex):
    Trials = ["wu_6H2_2","wu_6H2_3", "wu_12H2_1","wu_12H2_2","wu_12H2_3" ]
    base_import_path = f"D:\\DataFromE\\irsst\\{participant}\\"
    base_export_path = f"C:\\Users\\Assila\\Documents\\CEINMS_test\\ElaboratedData\\XmlGenerator\\{participant}\\Trials\\"

    for trial in Trials:
    # EMG and Force
        EMG_name = participant[0].upper()+participant[1:3]+participant[3].upper()+sex+trial.split('_',1)[1]+".sto"
        IK = "wu_"+participant[0].upper()+participant[1:3]+participant[3].upper()+sex+trial.split('_',1)[1]
        MA = "wu_"+participant[0].upper()+participant[1:3]+participant[3].upper()+sex+trial.split('_',1)[1]+"_MuscleAnalysis_"
        MAs = ["Length.sto", "MomentArm_Acromioclavicular_r1.sto", "MomentArm_Acromioclavicular_r2.sto", "MomentArm_Acromioclavicular_r3.sto",
              "MomentArm_sternoclavicular_r1.sto", "MomentArm_sternoclavicular_r2.sto",
              "MomentArm_shoulder_ele.sto", "MomentArm_shoulder_plane.sto", "MomentArm_shoulder_rotation.sto"]
        Ext_force = "Ex_"+participant[0].upper()+participant[1:3]+participant[3].upper()+sex+trial.split('_',1)[1]+".xml" 
        if os.path.isdir(base_export_path+trial):
            pass
        else:
            os.mkdir(base_export_path+trial)
        print(EMG_name)
        if os.path.exists(base_import_path+"0_emg\\"+EMG_name):
            copyfile(base_import_path+"0_emg\\"+EMG_name, base_export_path+trial+"\\EMG.sto")
            if os.path.exists(base_import_path+"0_forces\\"+EMG_name):
                copyfile(base_import_path+"0_forces\\"+EMG_name, base_export_path+trial+"\\Force.sto")
                if os.path.exists(base_import_path+"1_inverse_kinematic\\"+IK+".mot"):
                    copyfile(base_import_path+"1_inverse_kinematic\\"+IK+".mot", base_export_path+trial+"\\"+IK+".mot")
                    if os.path.exists(base_import_path+"2_inverse_dynamic\\"+IK+".sto"):
                        copyfile(base_import_path+"2_inverse_dynamic\\"+IK+".sto", base_export_path+trial+"\\"+IK+".sto")
                        for mf in MAs:
                            if os.path.exists(base_import_path+"4_muscle_analysis\\"+MA+mf):
                                if os.path.isdir(base_export_path+trial+"\\MA"):
                                    pass
                                else:
                                    os.mkdir(f"{base_export_path+trial}\\MA")#Todo Improve the loop
                                copyfile(base_import_path+"4_muscle_analysis\\"+MA+mf,base_export_path+trial+"\\MA\\"+MA+mf)
                            else:
                                print(f"Missing muscle analysis for {participant} at {trial}")
                                continue
                    else:
                        print(f"Missing inverse dynamic for {participant} at {trial}")
                        continue
                else:
                    print(f"Missing inverse kinematic for {participant} at {trial}")
                    continue
            else:
                print(f"Missing force for {participant} at {trial}")
                continue
        else:
            print(f"Missing EMG for {participant} at {trial}")
            continue
            
        if os.path.isdir(base_export_path+trial+"\\Ext_load"):
            pass
        else:
            os.mkdir(f"{base_export_path+trial}\\Ext_load")
    
#   Generate the external force xml
        ext_lo = osim.ExternalLoads()
        ext_lo.setExternalLoadsModelKinematicsFileName(f"{base_export_path+trial}\\{IK}.mot")
        ext_lo.setDataFileName(f"{base_export_path+trial}\\Force.sto")
        ext_lo.setLowpassCutoffFrequencyForLoadKinematics(5)
        ext_lo.printToXML(f"{base_export_path+trial}\\Ext_load\\Test{Ext_force}")
    
        ext_tree = etree.parse(f"{base_export_path+trial}\\Ext_load\\Test{Ext_force}")
        root = ext_tree.getroot()
        objects = root[0][0]
        Exter_force = etree.SubElement(objects, "ExternalForce")
        Exter_force.set("name", "s2h")
        sub = etree.SubElement(Exter_force, "applied_to_body")
        sub.text="hand_r"
        sub = etree.SubElement(Exter_force, "force_expressed_in_body")
        sub.text="sensor1"
        sub = etree.SubElement(Exter_force, "point_expressed_in_body")
        sub.text="sensor1"
        sub = etree.SubElement(Exter_force, "force_identifier")
        sub.text="F"
        sub = etree.SubElement(Exter_force, "torque_identifier")
        sub.text="M"
        write_xml(f"{base_export_path+trial}\\Ext_load\\{Ext_force}", ext_tree,
              file_type="Ext_load", pretty_print=True)
    
        
#   Generate the trial xml
        trial_tree = etree.Element('inputData', xmlns__COLON__xsi="http://www.w3.org/2001/XMLSchema-instance",
                                 xsi__COLON__noNamespaceSchemaLocation="inputData.xsd")
        etree.SubElement(trial_tree, "muscleTendonLengthFile").text = f".\\{trial}\\MA\\{MA+MAs[0]}"
        etree.SubElement(trial_tree, "excitationsFile").text=f".\\{trial}\\EMG.sto"
        branch = etree.SubElement(trial_tree, "momentArmsFiles")
        for mf in MAs[1:]:
            Sub = etree.SubElement(branch, "momentArmsFile")
            Sub.set("dofName", mf.split('_',1)[-1].split('.')[0])
            Sub.text = f".\\{trial}\\MA\\{MA+mf}"
    
        etree.SubElement(trial_tree, "externalTorquesFile").text=f".\\{trial}\\{IK}.sto"
        etree.SubElement(trial_tree, "motionFile").text=f".\\{trial}\\{IK}.mot"
        etree.SubElement(trial_tree, "externalLoadsFile").text=f".\\{trial}\\Ext_load\\{Ext_force}"
    
        trial_xml = f"{base_export_path}\\{trial}.xml"
        write_xml(trial_xml, trial_tree, file_type="Trial",pretty_print=True)

# Insert ID data in prescribed controller file and implement it in the reduced osim model

In [263]:
conf = pd.read_csv("_conf.csv", sep=',')
conff = conf.drop([4,6,7,10,17,22,28,29,32,33,34,35,37,38,39,41,42,45,47,48], axis=0).sort_values(['participant'], ascending=[1]).reset_index(drop=True)
conff.to_csv("_conf_red.csv", sep=',')

for i in range(conff.shape[0]):
    participant = conff.participant[i]

    if conff.participant[i]=="sylg":
        sex = "M"
    elif conff.sex[i]==1:
        sex = "H"
    elif conff.sex[i]==0:
        sex = "F"
    else:
        print("not sure of the gender")
        
    if conff.participant[i]=="anns":
        trial = "wu_6H2_2"
    else:
        trial = "wu_6H2_1"
    
    ID_file = participant[0].upper()+participant[1:-1]+participant[-1].upper()+sex+trial[3:]
    print(f"The ID file is {ID_file}")
#     Update_Controller(participant, ID_file, trial)
#     reduce_model(participant,trial)
    trial_load(participant, sex)
          

The ID file is AimQF6H2_1
AimQF6H2_2.sto
AimQF6H2_3.sto
AimQF12H2_1.sto
AimQF12H2_2.sto
AimQF12H2_3.sto
The ID file is AleBH6H2_1
AleBH6H2_2.sto
AleBH6H2_3.sto
AleBH12H2_1.sto
AleBH12H2_2.sto
AleBH12H2_3.sto
The ID file is AleFF6H2_1
AleFF6H2_2.sto
AleFF6H2_3.sto
AleFF12H2_1.sto
AleFF12H2_2.sto
AleFF12H2_3.sto
The ID file is AmeGF6H2_1
AmeGF6H2_2.sto
AmeGF6H2_3.sto
AmeGF12H2_1.sto
AmeGF12H2_2.sto
AmeGF12H2_3.sto
The ID file is AmiAF6H2_1
AmiAF6H2_2.sto
AmiAF6H2_3.sto
AmiAF12H2_1.sto
AmiAF12H2_2.sto
AmiAF12H2_3.sto
The ID file is AnnSF6H2_2
AnnSF6H2_2.sto
AnnSF6H2_3.sto
Missing force for anns at wu_6H2_3
AnnSF12H2_1.sto
AnnSF12H2_2.sto
AnnSF12H2_3.sto
The ID file is ArsTH6H2_1
ArsTH6H2_2.sto
ArsTH6H2_3.sto
ArsTH12H2_1.sto
ArsTH12H2_2.sto
ArsTH12H2_3.sto
The ID file is BenLH6H2_1
BenLH6H2_2.sto
BenLH6H2_3.sto
BenLH12H2_1.sto
BenLH12H2_2.sto
BenLH12H2_3.sto
The ID file is CamBF6H2_1
CamBF6H2_2.sto
CamBF6H2_3.sto
CamBF12H2_1.sto
CamBF12H2_2.sto
CamBF12H2_3.sto
The ID file is DamGH6H2_1
Dam

In [ ]:
# Create violinplot
# sns.violinplot(x = "time", y="AC1_res", data=ID)

sns.kdeplot(ID.AC1_res)

# Show the plot
plt.show()

In [211]:
print(root[0][0])
for node in root[0][0][0]:
    print(node)

<Element objects at 0x1f8cbc063c8>
<Element applied_to_body at 0x1f8c7630f88>
<Element force_expressed_in_body at 0x1f8cad0dd48>
<Element point_expressed_in_body at 0x1f8ca40de88>
<Element force_identifier at 0x1f8c7630f88>
<Element torque_identifier at 0x1f8ca44ecc8>
